In [ ]:
# input: patient records:
#         each record has the patient ID, race, female,  and the list of all visits seperated by #
#         each visit record  hase DaysToEvent, age, list of icd 9 codes and year
# output: patient visit records:
#       patientID, Race, Female, DaysToEvent, Age,year, list of icd9 codes entered at that visit,hot-code representation of Dxs 


In [ ]:
# enter the outfile name and  dx code to extract all patient who has it in any of the visits
# return the number of patients and the total nuber of vists for all
# enter the type of the encodeing (120, 1K)


In [2]:
import pandas as pd
import numpy as np
import time
import re
import os
from operator import add

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 50000)


In [12]:
#filename = "HCUP_2003_to_2011_Patient_to_Visits.csv"
#filename = "hcup_data\HCUP_2003_to_2011_Patient_to_Visits_withoutLine40.csv"

#outfile = "hcup_data\HCUP_2003_to_2011_Patient_visits_encoded120.csv"
icd = pd.read_csv("icd_list.csv", header = 0)

In [126]:
infile = "data_test.csv"
outfile = "patientDxs.csv"
start = os.getcwd()
infile_path = os.path.relpath('..//hcup_data//'+infile, start)
outfile_path = os.path.relpath('..//hcup_data//'+outfile, start)


In [127]:
def coding_120(i):
    if i[:1]=='V':
        return 0
    else:
        i = int(i[:3])
        code = np.where(( (i>=icd['min']) & (i<=icd['max'])))[0]
        if len(code)==0:
            return 119
        return code[0]+1

def coding_1K(i):
     if i[:1]=='V': 
        return 0
     else:
        return int(i[:3])
    
def one_hot(l,width):
    code = [0]*width 
    for i in l:
        x = [0]*width 
        x[i] = 1
        code = list( map(add, code, x) )
    return code 

In [140]:
def PatientHasDx(infile, outfile, disease, bunchsize = 1000):
    bunch = []
    counter = 0
    file_length = 0
    tic = time.perf_counter()
    with open(infile, "r") as r, open(outfile, "w") as w:
        for line in r:
            # each line is a patient recor with multiple visits seperated by #
            pat_bunch = []
            positive = False
            is_empty_visit = False
            empty_visits = 0
            counter +=1
            data = line.split("|")
            code1, code2 = data[1].split(",")
            dxlist_all = data[2].split("#")
            l = len(dxlist_all)
            # Going through each visit to make it a new record 
            for v in range(l):
                # extracting the list of icd codes
                dxlist = dxlist_all[v].split(",")
                year = dxlist[-1].replace("\n", "")
                codes = (dxlist[2:-2])
                # Calculating exact days between visists 
                if v ==0:
                    prev_visit = dxlist[0]
                # discart the if the visist doesn't have any diagnosis codes
                if len(codes)==0:
                    is_empty_visit = True
                    empty_visits += 1
                    continue
                codes_dx = [c[2:] for c in codes if (c[:2]=='D_') & (c[3:].isnumeric())] 
                if len(codes_dx)==0:
                    is_empty_visit = True
                    empty_visits += 1
                    continue
                dis_len = len(disease)
                if not positive: 
                    for s in codes_dx:
                        if disease in s[:dis_len]:
                            positive = True
                            break
                dx_hot_code = one_hot([coding_120(j) for j in codes_dx],120)
                pat_bunch.append(','.join((data[0],code1,code2,str(int(dxlist[0])-int(prev_visit)),dxlist[1],year,
                                           ' '.join(codes), str(dx_hot_code)[1:-1])) + '\n')
                prev_visit = dxlist[0]
            if positive :
                bunch = bunch + pat_bunch
                file_length += l
                if is_empty_visit: 
                    file_length -=empty_visits
            if counter == bunchsize:
                w.writelines(pat_bunch)
                print(time.perf_counter())
                bunch = []
                counter = 0
        w.writelines(bunch)
    toc = time.perf_counter()
    print(f"Processed patients who has {disease} in  {toc - tic:0.4f} seconds") 
    print("number of visit records is: ", str(file_length))


In [ ]:
PatientHasDx(infile_path, outfile_path, '40', bunchsize = 1000)


In [125]:
icd

,min,max,Label
0,1,9,Intestinal Infectious Diseases
1,10,18,Tuberculosis
2,20,27,Zoonotic Bacterial Diseases
3,30,41,Other Bacterial Diseases
4,42,42,Human Immunodeficiency Virus
5,45,49,Poliomyelitis And Other Non-Arthropod-Borne Vi...
6,50,59,Viral Diseases Accompanied By Exanthem
7,60,66,Arthropod-Borne Viral Diseases
8,70,79,Other Diseases Due To Viruses And Chlamydiae
9,80,88,Rickettsioses And Other Arthropod-Borne Diseases
